In [1]:
import pandas as pd
import csv
import os
import sys
import csv
import time
import numpy as np
import random
from scipy.io import arff

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, f"{os.path.dirname(os.getcwd())}/src")
from utils import evaluate_result
from data_imbalance_src.smote_oversampling import RandomOversampling, ADASYNOversampling, BorderlineSMOTEOversampling, SMOTEOversampling, SVMSMOTEOversampling
from data_imbalance_src.smote_oversampling import SMOTUNEDOversampling
from data_imbalance_src.dazzle import DAZZLEOversampling
from data_imbalance_src.dazzle1 import DAZZLE1Oversampling
from data_imbalance_src.dazzle2 import DAZZLE2Oversampling
from data_imbalance_src.Imbalance_Farou2022.data_generation import GANOversampling
from data_imbalance_src.random_projection import RandomProjectionOversampling

from diveplane.utilities import infer_feature_attributes
from diveplane.geminai import Geminai

from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer

2023-10-05 04:10:04.105912: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-05 04:10:04.110488: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 04:10:04.167770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-05 04:10:04.169217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 04:10:04.713653: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
rs_list = random.sample(range(50, 500), 10)

In [3]:
for repeat in range(7, 10):
    print(f"----- in repeat {repeat+1} -----")
    rs = rs_list[repeat]
    write_file = f"Moodle_Vuln_res_r{repeat+1}_rn{rs}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/{write_file}"
    with open(write_path, "w", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])
        
#     data_path = f"{os.path.dirname(os.getcwd())}/data/JavaScript_Vulnerability/"
#     datafiles = [f for f in os.listdir(data_path) if f.endswith("csv")]
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/Vulnerable_Files/moodle-2_0_0-metrics.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['IsVulnerable'] = df['IsVulnerable'].astype('str')
    d = {'b\'yes\'': 1, 'b\'no\'': 0}
    df['IsVulnerable'] = df['IsVulnerable'].astype(str).map(d).fillna(df['IsVulnerable'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))
    
    ########## normal run ##########
    # normal run - without any oversampling technique
    print("----- normal -----")

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train)
    clf_KNN.fit(X_train_scale, y_train)
    clf_LR.fit(X_train_scale, y_train)
    clf_DT.fit(X_train_scale, y_train)
    clf_RF.fit(X_train_scale, y_train)
    clf_LightGBM.fit(X_train_scale, y_train)
    clf_Adaboost.fit(X_train_scale, y_train)
    clf_GBDT.fit(X_train_scale, y_train)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["No", 0, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["No", 0, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["No", 0, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["No", 0, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["No", 0, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["No", 0, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["No", 0, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["No", 0, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## random run ##########
    # random oversampling run - random oversampling technique
    print("----- random -----")

    rt, X_train_new, y_train_new = RandomOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Random", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Random", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Random", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Random", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Random", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Random", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Random", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Random", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## ADASYN run ##########
    # ADASYN oversampling run - ADASYN oversampling technique
    print("----- ADASYN ------")

    rt, X_train_new, y_train_new = ADASYNOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["ADASYN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["ADASYN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["ADASYN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["ADASYN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["ADASYN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["ADASYN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["ADASYN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["ADASYN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## BorderlineSMOTE run ##########
    # BorderlineSMOTE oversampling run - BorderlineSMOTE oversampling technique
    print("----- borderlineSMOTE -----")

    rt, X_train_new, y_train_new = BorderlineSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["BorderlineSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTE run ##########
    # SMOTE oversampling run - SMOTE oversampling technique
    print("----- SMOTE -----")

    rt, X_train_new, y_train_new = SMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SVMSMOTE run ##########
    # SVMSMOTE oversampling run - SVMSMOTE oversampling technique
    print("----- SVMSMOTE -----")

    rt, X_train_new, y_train_new = SVMSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SVMSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTUNED run ##########
    # SMOTUNED oversampling run - SMOTUNED oversampling technique
    print("----- SMOTUNED -----")

    rt_SVM, X_train_new_SVM, y_train_new_SVM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="SVM")

    scaler = StandardScaler()
    X_train_scale_SVM = pd.DataFrame(scaler.fit_transform(X_train_new_SVM), columns=X_train_new_SVM.columns)
    X_test_scale_SVM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of SVM: 1:" + str(round(y_train_new_SVM.value_counts()[0] / y_train_new_SVM.value_counts()[1])))

    rt_KNN, X_train_new_KNN, y_train_new_KNN = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="KNN")

    scaler = StandardScaler()
    X_train_scale_KNN = pd.DataFrame(scaler.fit_transform(X_train_new_KNN), columns=X_train_new_KNN.columns)
    X_test_scale_KNN = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of KNN: 1:" + str(round(y_train_new_KNN.value_counts()[0] / y_train_new_KNN.value_counts()[1])))

    rt_LR, X_train_new_LR, y_train_new_LR = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="LR")

    scaler = StandardScaler()
    X_train_scale_LR = pd.DataFrame(scaler.fit_transform(X_train_new_LR), columns=X_train_new_LR.columns)
    X_test_scale_LR = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LR: 1:" + str(round(y_train_new_LR.value_counts()[0] / y_train_new_LR.value_counts()[1])))

    rt_DT, X_train_new_DT, y_train_new_DT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="DT")

    scaler = StandardScaler()
    X_train_scale_DT = pd.DataFrame(scaler.fit_transform(X_train_new_DT), columns=X_train_new_DT.columns)
    X_test_scale_DT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of DT: 1:" + str(round(y_train_new_DT.value_counts()[0] / y_train_new_DT.value_counts()[1])))

    rt_RF, X_train_new_RF, y_train_new_RF = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="RF")

    scaler = StandardScaler()
    X_train_scale_RF = pd.DataFrame(scaler.fit_transform(X_train_new_RF), columns=X_train_new_RF.columns)
    X_test_scale_RF = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of RF: 1:" + str(round(y_train_new_RF.value_counts()[0] / y_train_new_RF.value_counts()[1])))

    rt_LightGBM, X_train_new_LightGBM, y_train_new_LightGBM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="LightGBM")

    scaler = StandardScaler()
    X_train_scale_LightGBM = pd.DataFrame(scaler.fit_transform(X_train_new_LightGBM), columns=X_train_new_LightGBM.columns)
    X_test_scale_LightGBM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LightGBM: 1:" + str(round(y_train_new_LightGBM.value_counts()[0] / y_train_new_LightGBM.value_counts()[1])))

    rt_Adaboost, X_train_new_Adaboost, y_train_new_Adaboost = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="Adaboost")

    scaler = StandardScaler()
    X_train_scale_Adaboost = pd.DataFrame(scaler.fit_transform(X_train_new_Adaboost), columns=X_train_new_Adaboost.columns)
    X_test_scale_Adaboost = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of Adaboost: 1:" + str(round(y_train_new_Adaboost.value_counts()[0] / y_train_new_Adaboost.value_counts()[1])))

    rt_GBDT, X_train_new_GBDT, y_train_new_GBDT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                       y_train=y_train, y_test=y_test, model="GBDT")

    scaler = StandardScaler()
    X_train_scale_GBDT = pd.DataFrame(scaler.fit_transform(X_train_new_GBDT), columns=X_train_new_GBDT.columns)
    X_test_scale_GBDT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of GBDT: 1:" + str(round(y_train_new_GBDT.value_counts()[0] / y_train_new_GBDT.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale_SVM, y_train_new_SVM)
    clf_KNN.fit(X_train_scale_KNN, y_train_new_KNN)
    clf_LR.fit(X_train_scale_LR, y_train_new_LR)
    clf_DT.fit(X_train_scale_DT, y_train_new_DT)
    clf_RF.fit(X_train_scale_RF, y_train_new_RF)
    clf_LightGBM.fit(X_train_scale_LightGBM, y_train_new_LightGBM)
    clf_Adaboost.fit(X_train_scale_Adaboost, y_train_new_Adaboost)
    clf_GBDT.fit(X_train_scale_GBDT, y_train_new_GBDT)

    y_pred_SVM = clf_SVM.predict(X_test_scale_SVM)
    y_pred_KNN = clf_KNN.predict(X_test_scale_KNN)
    y_pred_LR = clf_LR.predict(X_test_scale_LR)
    y_pred_DT = clf_DT.predict(X_test_scale_DT)
    y_pred_RF = clf_RF.predict(X_test_scale_RF)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale_LightGBM)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale_Adaboost)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale_GBDT)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTUNED", rt_SVM, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_KNN, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LR, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTUNED", rt_DT, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTUNED", rt_RF, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LightGBM, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_Adaboost, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTUNED", rt_GBDT, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = DAZZLEOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## WGAN run ##########
    # WGAN oversampling run - WGAN oversampling technique
    print("----- WGAN -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = GANOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["WGAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["WGAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["WGAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["WGAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["WGAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["WGAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["WGAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["WGAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))


    ########## RandomProjection run ##########
    # Random projection oversampling run - Random projection oversampling technique
    print("----- Random Projection -----")
    X_train_RP = X_train.copy()
    y_train_RP = y_train.copy()
    rt, X_train_new, y_train_new = RandomProjectionOversampling(X_train=X_train_RP, y_train=y_train_RP)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["RP", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["RP", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["RP", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["RP", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["RP", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["RP", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["RP", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["RP", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 1 run ##########
    print("----- diveplane 1 -----")
    tar = y_train.name
    conditions = [{tar: 1},
                  {tar: 0}] * (int(X_train.shape[0] / 2))
    
    X_train_diveplane_1 = X_train.copy()
    y_train_diveplane_1 = y_train.copy()
    X_train_diveplane_1[tar] = y_train_diveplane_1
    
    features = infer_feature_attributes(X_train_diveplane_1)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_1, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = gen_df.iloc[:, :-1]
    y_train_new = gen_df.iloc[:, -1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 2 run ##########
    print("----- diveplane 2 -----")
    tar = y_train.name
    n_cases_diff = y_train.value_counts()[0] - y_train.value_counts()[1]
    conditions = [{tar: 1}] * n_cases_diff
    
    X_train_diveplane_2 = X_train.copy()
    y_train_diveplane_2 = y_train.copy()
    X_train_diveplane_2[tar] = y_train_diveplane_2
    
    features = infer_feature_attributes(X_train_diveplane_2)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_2, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = pd.concat([X_train_diveplane_2, gen_df], ignore_index=True, axis=0)
    y_train_new = X_train_new.iloc[:, -1]
    X_train_new = X_train_new.iloc[:, :-1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DS run ##########
    print("----- DS -----")
    mode = "independent_attribute_mode"

    col = X_train.columns
    tar = y_train.name
    
    X_train_DS = X_train.copy()
    y_train_DS = y_train.copy()
    X_train_DS[tar] = y_train_DS
    write_df = X_train_DS[X_train_DS[tar] == 1]
    write_df = write_df.iloc[:, :-1]
    write_df.to_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv", index=False)
    
    threshold = 20
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

    start_time = time.time()

    description_file = f"{os.path.dirname(os.getcwd())}/extra/js_vuln.json"
    describer = DataDescriber(category_threshold=threshold)
    describer.describe_dataset_in_independent_attribute_mode(
        dataset_file=f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv"
    )
    describer.save_dataset_description_to_file(description_file)

    generator = DataGenerator()
    generator.generate_dataset_in_independent_mode(num_tuples_to_generate, description_file)
    generator.save_synthetic_data(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv")

    rt = time.time() - start_time

    X_train_new = pd.read_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv").to_numpy()
    y_train_new = np.ones(num_tuples_to_generate)
    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DS", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DS", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DS", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DS", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DS", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DS", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DS", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DS", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV FASTML run ##########
    print("----- SDV FASTML -----")
    col = X_train.columns
    tar = y_train.name
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])
    
    X_train_SDV = X_train.copy()
    y_train_SDV = y_train.copy()
    X_train_SDV[tar] = y_train_SDV
    pos_df = X_train_SDV[X_train_SDV[tar] == 1]
    pos_df = pos_df.iloc[:, :-1]

    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=pos_df)
    
    start_time = time.time()
    syn1 = SingleTablePreset(metadata, name="FAST_ML")
    syn1.fit(data=pos_df)
    X_train_new = syn1.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_FASTML", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GC ##########
    print("----- SDV GC -----")
    start_time = time.time()
    syn2 = GaussianCopulaSynthesizer(metadata)
    syn2.fit(data=pos_df)
    X_train_new = syn2.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GC", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GC", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GC", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GC", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GAN ##########
    print("----- SDV GAN -----")
    start_time = time.time()
    syn3 = CTGANSynthesizer(metadata)
    syn3.fit(data=pos_df)
    X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

----- in repeat 8 -----
y value counts: 
 0    2032
1      24
Name: IsVulnerable, dtype: int64
y class ratio: 1: 85
--- y train classes count: 
0    1625
1      19
Name: IsVulnerable, dtype: int64
--- y train ratio: 1:86
 
--- y test classes count: 
0    407
1      5
Name: IsVulnerable, dtype: int64
--- y test ratio: 1:81
----- normal -----
----- random -----
y train ratio: 1:1
----- ADASYN ------
y train ratio: 1:1
----- borderlineSMOTE -----
y train ratio: 1:1
----- SMOTE -----
y train ratio: 1:1
----- SVMSMOTE -----
y train ratio: 1:3
----- SMOTUNED -----
y train ratio of SVM: 1:1
y train ratio of KNN: 1:1
y train ratio of LR: 1:1
y train ratio of DT: 1:1
y train ratio of RF: 1:1
y train ratio of LightGBM: 1:1
y train ratio of Adaboost: 1:1
y train ratio of GBDT: 1:1
----- DAZZLE -----
  0%|                                                                           | 0/100 [00:00<?, ?trial/s, best loss=?]

2023-09-21 19:56:04.211615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-21 19:56:04.212326: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


100%|██████████████████████████████████████████████| 100/100 [17:49<00:00, 10.69s/trial, best loss: -0.9941194676570722]
Best Hyperparameters: {'batch_size': 16, 'discriminator_activation_fn': <function leaky_relu at 0x7f3b0a937160>, 'discriminator_layer_normalization': False, 'discriminator_lr': 0.017453678213581802, 'discriminator_optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>, 'epochs': 361, 'generator_activation_fn': <function leaky_relu at 0x7f3b0a937160>, 'generator_layer_normalization': False, 'generator_lr': 0.06272919360230805, 'generator_optimizer': <class 'keras.src.optimizers.rmsprop.RMSprop'>}
Best G-Measure: 0.9941194676570722
22/52 [===========>..................] - ETA: 0s

/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


52/52 [==============================] - 0s 2ms/step
y train ratio: 1:1
----- WGAN -----
MAX CLASS 1625
CLASS ID 1
Epoch 1/150 completed. Gen loss: -0.004087403416633606. Desc loss_real: 0.011342313140630722. Desc loss_fake: 0.004087403416633606
Epoch 51/150 completed. Gen loss: -0.008132525719702244. Desc loss_real: 0.011927622370421886. Desc loss_fake: 0.008132525719702244
Epoch 101/150 completed. Gen loss: -0.007668113335967064. Desc loss_real: 0.008749130181968212. Desc loss_fake: 0.007668113335967064


...

y train ratio: 1:1
----- diveplane 2 -----


/home/lingxiao/diveplane/lib/python3.8/site-packages/diveplane/geminai/base.py:376: UserWarning: The value of the parameter `generate_new_cases` is set as 'no' and not 'always', which means it is possible that some cases that are very similar to the original data may be returned. To prevent this from happening, please set generate_new_cases to 'always'.
  warnings.warn(


y train ratio: 1:1
----- DS -----
y train ratio: 1:1
----- SDV FASTML -----
y train ratio: 1:1
----- SDV GC -----
y train ratio: 1:1
----- SDV GAN -----


/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1


In [2]:
rnl = [381, 132, 485, 273, 478, 111, 469, 382, 166, 273]

for idx, rn in enumerate(rnl):
    write_file = f"Moodle_Vuln_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Moodle_Vuln/{write_file}"
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/Vulnerable_Files/moodle-2_0_0-metrics.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['IsVulnerable'] = df['IsVulnerable'].astype('str')
    d = {'b\'yes\'': 1, 'b\'no\'': 0}
    df['IsVulnerable'] = df['IsVulnerable'].astype(str).map(d).fillna(df['IsVulnerable'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    2032
1      24
Name: IsVulnerable, dtype: int64
y class ratio: 1: 85
--- y train classes count: 
0    1625
1      19
Name: IsVulnerable, dtype: int64
--- y train ratio: 1:86
 
--- y test classes count: 
0    407
1      5
Name: IsVulnerable, dtype: int64
--- y test ratio: 1:81
----- DAZZLE1 -----
[    0/3562] LG:-0.789 LD:-1.031 D:-0.143 GP:0.099 RMSEAVG:0.295 NUM:0.295 SynTraiAuc:0.773 RFAcc:1.000                 
[  500/3562] LG:-0.287 LD:0.424 D:0.448 GP:0.003 RMSEAVG:0.295 NUM:0.295 SynTraiAuc:0.773 RFAcc:1.000                   
[ 1000/3562] LG:-0.367 LD:0.104 D:0.144 GP:0.004 RMSEAVG:0.295 NUM:0.295 SynTraiAuc:0.773 RFAcc:1.000                   
[ 1500/3562] LG:-0.434 LD:0.131 D:0.180 GP:0.005 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.773 RFAcc:0.951                   
[ 2000/3562] LG:-0.389 LD:0.098 D:0.120 GP:0.002 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.773 RFAcc:0.951                   
[ 2500/3562] LG:-0.308 LD:0.135 D:0.151 GP:0.002 RMSEAVG:0.035 NUM:0.035 Sy

[15000/15288] LG:0.561 LD:-0.195 D:-0.163 GP:0.004 RMSEAVG:0.088 NUM:0.088 SynTraiAuc:0.456 RFAcc:0.999                 
[15288/15288] LG:0.650 LD:0.115 D:0.159 GP:0.005 RMSEAVG:0.088 NUM:0.088 SynTraiAuc:0.456 RFAcc:0.999                   
needed sample                                                                                                           
1606                                                                                                                    
[    0/19812] LG:0.043 LD:-3.225 D:-0.135 GP:0.343 RMSEAVG:0.367 NUM:0.367 SynTraiAuc:0.506 RFAcc:1.000                 
[  500/19812] LG:-0.580 LD:-0.245 D:-0.154 GP:0.010 RMSEAVG:0.367 NUM:0.367 SynTraiAuc:0.506 RFAcc:1.000                
[ 1000/19812] LG:0.537 LD:1.020 D:1.087 GP:0.007 RMSEAVG:0.367 NUM:0.367 SynTraiAuc:0.506 RFAcc:1.000                   
[ 1500/19812] LG:0.593 LD:0.667 D:0.888 GP:0.025 RMSEAVG:0.519 NUM:0.519 SynTraiAuc:0.506 RFAcc:1.000                   
[ 2000/19812] LG:-0.387 LD:-0.42

[ 1500/2730] LG:-0.061 LD:0.105 D:0.151 GP:0.003 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.945 RFAcc:0.920                   
[ 2000/2730] LG:-0.381 LD:0.029 D:0.150 GP:0.008 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.945 RFAcc:0.920                   
[ 2500/2730] LG:-0.323 LD:0.112 D:0.179 GP:0.004 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.945 RFAcc:0.941                   
[ 2730/2730] LG:-0.261 LD:0.010 D:0.063 GP:0.003 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.945 RFAcc:0.941                   
needed sample                                                                                                           
1606                                                                                                                    
[    0/10140] LG:0.119 LD:-2.468 D:0.010 GP:0.124 RMSEAVG:0.175 NUM:0.175 SynTraiAuc:0.221 RFAcc:1.000                  
[  500/10140] LG:-0.253 LD:-0.324 D:-0.166 GP:0.008 RMSEAVG:0.175 NUM:0.175 SynTraiAuc:0.221 RFAcc:1.000                
[ 1000/10140] LG:-0.437 LD:-0.63

[13000/15808] LG:-0.567 LD:-0.201 D:-0.185 GP:0.002 RMSEAVG:0.102 NUM:0.102 SynTraiAuc:0.997 RFAcc:0.919                
[13500/15808] LG:-0.676 LD:-0.253 D:-0.245 GP:0.001 RMSEAVG:0.102 NUM:0.102 SynTraiAuc:0.997 RFAcc:0.919                
[14000/15808] LG:-0.607 LD:0.041 D:0.088 GP:0.005 RMSEAVG:0.112 NUM:0.112 SynTraiAuc:0.997 RFAcc:0.912                  
[14500/15808] LG:-0.700 LD:-0.081 D:-0.051 GP:0.003 RMSEAVG:0.112 NUM:0.112 SynTraiAuc:0.997 RFAcc:0.912                
[15000/15808] LG:-0.772 LD:-0.190 D:-0.058 GP:0.013 RMSEAVG:0.111 NUM:0.111 SynTraiAuc:0.635 RFAcc:0.974                
[15500/15808] LG:-0.804 LD:0.011 D:0.031 GP:0.002 RMSEAVG:0.111 NUM:0.111 SynTraiAuc:0.635 RFAcc:0.974                  
[15808/15808] LG:-0.723 LD:0.012 D:0.032 GP:0.002 RMSEAVG:0.111 NUM:0.111 SynTraiAuc:0.635 RFAcc:0.974                  
needed sample                                                                                                           
1606                            

[ 6000/31827] LG:-0.163 LD:0.066 D:0.090 GP:0.003 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.936 RFAcc:0.916                  
[ 6500/31827] LG:-0.099 LD:0.082 D:0.108 GP:0.003 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.936 RFAcc:0.919                  
[ 7000/31827] LG:-0.048 LD:0.014 D:0.023 GP:0.001 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.936 RFAcc:0.919                  
[ 7500/31827] LG:-0.089 LD:0.006 D:0.034 GP:0.003 RMSEAVG:0.046 NUM:0.046 SynTraiAuc:0.936 RFAcc:0.886                  
[ 8000/31827] LG:-0.002 LD:0.052 D:0.061 GP:0.001 RMSEAVG:0.046 NUM:0.046 SynTraiAuc:0.936 RFAcc:0.886                  
[ 8500/31827] LG:-0.132 LD:0.011 D:0.030 GP:0.002 RMSEAVG:0.046 NUM:0.046 SynTraiAuc:0.936 RFAcc:0.886                  
[ 9000/31827] LG:-0.107 LD:0.037 D:0.054 GP:0.002 RMSEAVG:0.051 NUM:0.051 SynTraiAuc:0.936 RFAcc:0.835                  
[ 9500/31827] LG:-0.103 LD:0.062 D:0.089 GP:0.003 RMSEAVG:0.051 NUM:0.051 SynTraiAuc:0.936 RFAcc:0.835                  
[10000/31827] LG:0.033 LD:-0.008

...

[ 2000/2600] LG:0.032 LD:-0.034 D:0.073 GP:0.008 RMSEAVG:0.075 NUM:0.075 SynTraiAuc:0.328 RFAcc:0.995                   
[ 2500/2600] LG:0.078 LD:0.063 D:0.137 GP:0.006 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.328 RFAcc:0.857                    
[ 2600/2600] LG:0.119 LD:0.061 D:0.099 GP:0.003 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.328 RFAcc:0.857                    
needed sample                                                                                                           
1606                                                                                                                    
[    0/6708] LG:0.255 LD:-0.517 D:-0.037 GP:0.060 RMSEAVG:0.170 NUM:0.170 SynTraiAuc:0.877 RFAcc:0.990                  
[  500/6708] LG:0.362 LD:0.107 D:0.169 GP:0.008 RMSEAVG:0.170 NUM:0.170 SynTraiAuc:0.877 RFAcc:0.990                    
[ 1000/6708] LG:0.292 LD:0.085 D:0.154 GP:0.009 RMSEAVG:0.170 NUM:0.170 SynTraiAuc:0.877 RFAcc:0.990                    
[ 1500/6708] LG:0.350 LD:0.205 D

[ 5000/18356] LG:-1.307 LD:-0.116 D:-0.108 GP:0.001 RMSEAVG:0.101 NUM:0.101 SynTraiAuc:0.245 RFAcc:0.967                
[ 5500/18356] LG:-1.200 LD:0.040 D:0.054 GP:0.002 RMSEAVG:0.101 NUM:0.101 SynTraiAuc:0.245 RFAcc:0.967                  
[ 6000/18356] LG:-1.472 LD:-0.143 D:-0.135 GP:0.001 RMSEAVG:0.101 NUM:0.101 SynTraiAuc:0.245 RFAcc:0.967                
[ 6500/18356] LG:-1.315 LD:-0.042 D:-0.022 GP:0.002 RMSEAVG:0.085 NUM:0.085 SynTraiAuc:0.245 RFAcc:0.970                
[ 7000/18356] LG:-1.651 LD:-0.076 D:-0.035 GP:0.005 RMSEAVG:0.085 NUM:0.085 SynTraiAuc:0.245 RFAcc:0.970                
[ 7500/18356] LG:-1.726 LD:-0.280 D:-0.177 GP:0.013 RMSEAVG:0.116 NUM:0.116 SynTraiAuc:0.245 RFAcc:1.000                
[ 8000/18356] LG:-1.430 LD:-0.257 D:-0.249 GP:0.001 RMSEAVG:0.116 NUM:0.116 SynTraiAuc:0.245 RFAcc:1.000                
[ 8500/18356] LG:-1.342 LD:0.362 D:0.378 GP:0.002 RMSEAVG:0.116 NUM:0.116 SynTraiAuc:0.245 RFAcc:1.000                  
[ 9000/18356] LG:-1.446 LD:0.009

[ 4000/31003] LG:-0.105 LD:-0.027 D:0.022 GP:0.003 RMSEAVG:0.086 NUM:0.086 SynTraiAuc:0.077 RFAcc:0.967                 
[ 4500/31003] LG:-0.077 LD:0.010 D:0.086 GP:0.005 RMSEAVG:0.086 NUM:0.086 SynTraiAuc:0.077 RFAcc:0.967                  
[ 5000/31003] LG:-0.097 LD:0.062 D:0.195 GP:0.010 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.969 RFAcc:0.889                  
[ 5500/31003] LG:-0.030 LD:0.089 D:0.125 GP:0.003 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.969 RFAcc:0.889                  
[ 6000/31003] LG:0.009 LD:0.087 D:0.138 GP:0.004 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.969 RFAcc:0.889                   
[ 6500/31003] LG:-0.071 LD:0.089 D:0.109 GP:0.001 RMSEAVG:0.060 NUM:0.060 SynTraiAuc:0.969 RFAcc:0.834                  
[ 7000/31003] LG:-0.045 LD:0.097 D:0.123 GP:0.002 RMSEAVG:0.060 NUM:0.060 SynTraiAuc:0.969 RFAcc:0.834                  
[ 7500/31003] LG:0.017 LD:-0.031 D:0.019 GP:0.004 RMSEAVG:0.090 NUM:0.090 SynTraiAuc:0.969 RFAcc:0.818                  
[ 8000/31003] LG:0.009 LD:0.070 

[ 1500/11180] LG:-0.243 LD:0.152 D:0.162 GP:0.001 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.433 RFAcc:0.926                  
[ 2000/11180] LG:-0.286 LD:0.073 D:0.124 GP:0.003 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.433 RFAcc:0.926                  
[ 2500/11180] LG:-0.238 LD:0.081 D:0.130 GP:0.003 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.433 RFAcc:0.930                  
[ 3000/11180] LG:-0.247 LD:0.067 D:0.151 GP:0.005 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.433 RFAcc:0.930                  
[ 3500/11180] LG:-0.003 LD:0.124 D:0.199 GP:0.005 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.433 RFAcc:0.930                  
[ 4000/11180] LG:-0.388 LD:-0.038 D:0.042 GP:0.005 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.433 RFAcc:0.952                 
[ 4500/11180] LG:-0.363 LD:0.146 D:0.169 GP:0.001 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.433 RFAcc:0.952                  
[ 5000/11180] LG:-0.129 LD:0.019 D:0.123 GP:0.006 RMSEAVG:0.083 NUM:0.083 SynTraiAuc:0.589 RFAcc:0.985                  
[ 5500/11180] LG:-0.040 LD:0.016

[ 6000/7618] LG:0.693 LD:0.139 D:0.155 GP:0.002 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.669 RFAcc:0.925                    
[ 6500/7618] LG:0.770 LD:0.176 D:0.204 GP:0.003 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.669 RFAcc:0.903                    
[ 7000/7618] LG:0.785 LD:0.090 D:0.108 GP:0.002 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.669 RFAcc:0.903                    
[ 7500/7618] LG:0.814 LD:0.110 D:0.126 GP:0.001 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.669 RFAcc:0.866                    
[ 7618/7618] LG:0.820 LD:0.183 D:0.203 GP:0.002 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.669 RFAcc:0.866                    
needed sample                                                                                                           
1606                                                                                                                    
100%|██████████████████████████████████████████████████████████████████| 5/5 [02:44<00:00, 32.95s/trial, best loss: 1.0]
[ 5000/11180] LG:0.455 LD:0.029 

In [3]:
rnl = [381, 132, 485, 273, 478, 111, 469, 382, 166, 273]

for idx, rn in enumerate(rnl):
    write_file = f"Moodle_Vuln_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Moodle_Vuln/{write_file}"
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/Vulnerable_Files/moodle-2_0_0-metrics.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['IsVulnerable'] = df['IsVulnerable'].astype('str')
    d = {'b\'yes\'': 1, 'b\'no\'': 0}
    df['IsVulnerable'] = df['IsVulnerable'].astype(str).map(d).fillna(df['IsVulnerable'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE2 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE2Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)
    # scale data
    # scaler = StandardScaler()
    # X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    # X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    2032
1      24
Name: IsVulnerable, dtype: int64
y class ratio: 1: 85
--- y train classes count: 
0    1625
1      19
Name: IsVulnerable, dtype: int64
--- y train ratio: 1:86
 
--- y test classes count: 
0    407
1      5
Name: IsVulnerable, dtype: int64
--- y test ratio: 1:81
----- DAZZLE2 -----
[    0/23936] LG:-0.047 LD:-0.338 D:0.228 GP:0.038 RMSEAVG:0.260 NUM:0.260 SynTraiAuc:0.130 RFAcc:1.000                 
[  500/23936] LG:-0.642 LD:0.243 D:0.261 GP:0.001 RMSEAVG:0.260 NUM:0.260 SynTraiAuc:0.130 RFAcc:1.000                  
[ 1000/23936] LG:-0.585 LD:-0.064 D:0.133 GP:0.013 RMSEAVG:0.260 NUM:0.260 SynTraiAuc:0.130 RFAcc:1.000                 
[ 1500/23936] LG:-0.322 LD:0.104 D:0.217 GP:0.007 RMSEAVG:0.069 NUM:0.069 SynTraiAuc:0.130 RFAcc:0.940                  
[ 2000/23936] LG:-0.521 LD:0.088 D:0.130 GP:0.003 RMSEAVG:0.069 NUM:0.069 SynTraiAuc:0.130 RFAcc:0.940                  
[ 2500/23936] LG:-0.332 LD:0.111 D:0.142 GP:0.002 RMSEAVG:0.050 NUM:0.050 S

[ 4000/34672] LG:-0.152 LD:0.022 D:0.063 GP:0.005 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.460 RFAcc:0.882                  
[ 4500/34672] LG:-0.024 LD:-0.008 D:0.009 GP:0.002 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.460 RFAcc:0.882                 
[ 5000/34672] LG:0.014 LD:0.021 D:0.039 GP:0.002 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.315 RFAcc:0.962                   
[ 5500/34672] LG:-0.010 LD:0.086 D:0.092 GP:0.001 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.315 RFAcc:0.962                  
[ 6000/34672] LG:-0.019 LD:0.217 D:0.239 GP:0.003 RMSEAVG:0.058 NUM:0.058 SynTraiAuc:0.315 RFAcc:0.962                  
[ 6500/34672] LG:0.001 LD:0.181 D:0.258 GP:0.010 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.315 RFAcc:0.847                   
[ 7000/34672] LG:-0.006 LD:0.055 D:0.073 GP:0.002 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.315 RFAcc:0.847                  
[ 7500/34672] LG:0.041 LD:0.036 D:0.056 GP:0.003 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.315 RFAcc:0.863                   
[ 8000/34672] LG:-0.030 LD:0.168

[ 1000/11616] LG:0.052 LD:0.025 D:0.049 GP:0.005 RMSEAVG:0.204 NUM:0.204 SynTraiAuc:0.496 RFAcc:0.996                   
[ 1500/11616] LG:0.158 LD:0.052 D:0.106 GP:0.011 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.496 RFAcc:0.968                   
[ 2000/11616] LG:0.306 LD:0.088 D:0.123 GP:0.007 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.496 RFAcc:0.968                   
[ 2500/11616] LG:0.204 LD:0.057 D:0.074 GP:0.003 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.496 RFAcc:0.856                   
[ 3000/11616] LG:0.229 LD:0.103 D:0.165 GP:0.012 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.496 RFAcc:0.856                   
[ 3500/11616] LG:0.252 LD:0.087 D:0.108 GP:0.004 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.496 RFAcc:0.856                   
[ 4000/11616] LG:0.277 LD:-0.059 D:0.226 GP:0.057 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.496 RFAcc:0.913                  
[ 4500/11616] LG:0.194 LD:0.043 D:0.082 GP:0.008 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.496 RFAcc:0.913                   
[ 5000/11616] LG:0.102 LD:-0.003

1365                                                                                                                    
[    0/33704] LG:0.090 LD:-1.972 D:-0.084 GP:0.157 RMSEAVG:0.195 NUM:0.195 SynTraiAuc:0.967 RFAcc:0.982                 
[  500/33704] LG:0.258 LD:-0.154 D:-0.122 GP:0.003 RMSEAVG:0.195 NUM:0.195 SynTraiAuc:0.967 RFAcc:0.982                 
[ 1000/33704] LG:0.393 LD:0.047 D:0.100 GP:0.004 RMSEAVG:0.195 NUM:0.195 SynTraiAuc:0.967 RFAcc:0.982                   
[ 1500/33704] LG:0.767 LD:0.048 D:0.169 GP:0.010 RMSEAVG:0.068 NUM:0.068 SynTraiAuc:0.967 RFAcc:0.953                   
[ 2000/33704] LG:0.723 LD:0.010 D:0.117 GP:0.009 RMSEAVG:0.068 NUM:0.068 SynTraiAuc:0.967 RFAcc:0.953                   
[ 2500/33704] LG:0.526 LD:0.136 D:0.161 GP:0.002 RMSEAVG:0.052 NUM:0.052 SynTraiAuc:0.967 RFAcc:0.957                   
[ 3000/33704] LG:0.625 LD:0.109 D:0.177 GP:0.006 RMSEAVG:0.052 NUM:0.052 SynTraiAuc:0.967 RFAcc:0.957                   
[ 3500/33704] LG:0.768 LD:0.071 

[33000/33704] LG:0.787 LD:0.018 D:0.026 GP:0.001 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.710 RFAcc:0.788                   
[33500/33704] LG:0.840 LD:0.039 D:0.045 GP:0.001 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.710 RFAcc:0.788                   
[33704/33704] LG:0.773 LD:0.095 D:0.097 GP:0.000 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.710 RFAcc:0.788                   
needed sample                                                                                                           
1365                                                                                                                    
[    0/3696] LG:0.288 LD:-0.073 D:0.012 GP:0.014 RMSEAVG:0.358 NUM:0.358 SynTraiAuc:0.196 RFAcc:1.000                   
[  500/3696] LG:0.391 LD:0.191 D:0.214 GP:0.004 RMSEAVG:0.358 NUM:0.358 SynTraiAuc:0.196 RFAcc:1.000                    
[ 1000/3696] LG:0.264 LD:0.164 D:0.188 GP:0.004 RMSEAVG:0.358 NUM:0.358 SynTraiAuc:0.196 RFAcc:1.000                    
[ 1500/3696] LG:0.621 LD:0.183 D

...

[ 6500/13816] LG:-1.159 LD:-0.343 D:-0.298 GP:0.003 RMSEAVG:0.107 NUM:0.107 SynTraiAuc:0.783 RFAcc:0.975                
[ 7000/13816] LG:-1.148 LD:-0.507 D:-0.487 GP:0.001 RMSEAVG:0.107 NUM:0.107 SynTraiAuc:0.783 RFAcc:0.975                
[ 7500/13816] LG:-1.010 LD:-0.360 D:-0.351 GP:0.001 RMSEAVG:0.151 NUM:0.151 SynTraiAuc:0.783 RFAcc:0.978                
[ 8000/13816] LG:-1.223 LD:-0.471 D:-0.446 GP:0.002 RMSEAVG:0.151 NUM:0.151 SynTraiAuc:0.783 RFAcc:0.978                
[ 8500/13816] LG:-1.031 LD:-0.194 D:-0.146 GP:0.003 RMSEAVG:0.151 NUM:0.151 SynTraiAuc:0.783 RFAcc:0.978                
[ 9000/13816] LG:-1.106 LD:-0.211 D:-0.060 GP:0.009 RMSEAVG:0.165 NUM:0.165 SynTraiAuc:0.783 RFAcc:0.987                
[ 9500/13816] LG:-1.105 LD:-0.142 D:-0.040 GP:0.006 RMSEAVG:0.165 NUM:0.165 SynTraiAuc:0.783 RFAcc:0.987                
[10000/13816] LG:-1.179 LD:-0.169 D:-0.167 GP:0.000 RMSEAVG:0.143 NUM:0.143 SynTraiAuc:0.652 RFAcc:0.985                
[10500/13816] LG:-1.297 LD:-0.19

[16500/23408] LG:0.870 LD:0.201 D:0.209 GP:0.001 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.859 RFAcc:0.822                   
[17000/23408] LG:0.803 LD:0.005 D:0.087 GP:0.007 RMSEAVG:0.042 NUM:0.042 SynTraiAuc:0.859 RFAcc:0.822                   
[17500/23408] LG:0.800 LD:0.135 D:0.145 GP:0.001 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.859 RFAcc:0.781                   
[18000/23408] LG:0.931 LD:0.097 D:0.107 GP:0.001 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.859 RFAcc:0.781                   
[18500/23408] LG:0.866 LD:0.090 D:0.096 GP:0.001 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.859 RFAcc:0.781                   
[19000/23408] LG:0.820 LD:0.119 D:0.141 GP:0.002 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.859 RFAcc:0.818                   
[19500/23408] LG:0.925 LD:0.077 D:0.112 GP:0.003 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.859 RFAcc:0.818                   
[20000/23408] LG:0.951 LD:0.079 D:0.110 GP:0.003 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.774 RFAcc:0.771                   
[20500/23408] LG:0.845 LD:0.068 

[11836/11836] LG:0.494 LD:0.045 D:0.068 GP:0.003 RMSEAVG:0.078 NUM:0.078 SynTraiAuc:0.728 RFAcc:0.957                   
needed sample                                                                                                           
1363                                                                                                                    
[    0/4015] LG:0.456 LD:-6.744 D:-0.028 GP:0.353 RMSEAVG:0.268 NUM:0.268 SynTraiAuc:0.214 RFAcc:0.999                  
[  500/4015] LG:0.604 LD:0.149 D:0.182 GP:0.002 RMSEAVG:0.268 NUM:0.268 SynTraiAuc:0.214 RFAcc:0.999                    
[ 1000/4015] LG:0.668 LD:0.100 D:0.197 GP:0.005 RMSEAVG:0.268 NUM:0.268 SynTraiAuc:0.214 RFAcc:0.999                    
[ 1500/4015] LG:0.614 LD:-0.000 D:0.110 GP:0.006 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.214 RFAcc:0.929                   
[ 2000/4015] LG:0.703 LD:0.117 D:0.171 GP:0.003 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.214 RFAcc:0.929                    
[ 2500/4015] LG:0.668 LD:0.103 D

[ 1500/2354] LG:0.066 LD:0.057 D:0.123 GP:0.004 RMSEAVG:0.112 NUM:0.112 SynTraiAuc:0.748 RFAcc:0.990                    
[ 2000/2354] LG:0.068 LD:-0.024 D:0.075 GP:0.006 RMSEAVG:0.112 NUM:0.112 SynTraiAuc:0.748 RFAcc:0.990                   
[ 2354/2354] LG:-0.110 LD:-0.030 D:0.067 GP:0.006 RMSEAVG:0.112 NUM:0.112 SynTraiAuc:0.748 RFAcc:0.990                  
needed sample                                                                                                           
1367                                                                                                                    
100%|██████████████████████████████████████████████████████████████████| 5/5 [01:35<00:00, 19.12s/trial, best loss: 1.0]
[ 3315/3315] LG:0.085 LD:0.048 D:0.129 GP:0.005 RMSEAVG:0.072 NUM:0.072 SynTraiAuc:0.525 RFAcc:0.933     
needed sample 1606
y train ratio: 1:1
y value counts: 
 0    2032
1      24
Name: IsVulnerable, dtype: int64
y class ratio: 1: 85
--- y train classes count: 
0    1625


[ 6000/12892] LG:-0.366 LD:0.014 D:0.042 GP:0.002 RMSEAVG:0.054 NUM:0.054 SynTraiAuc:0.924 RFAcc:0.772                  
[ 6500/12892] LG:-0.355 LD:0.025 D:0.062 GP:0.002 RMSEAVG:0.080 NUM:0.080 SynTraiAuc:0.924 RFAcc:0.812                  
[ 7000/12892] LG:-0.256 LD:0.034 D:0.064 GP:0.002 RMSEAVG:0.080 NUM:0.080 SynTraiAuc:0.924 RFAcc:0.812                  
[ 7500/12892] LG:-0.290 LD:0.021 D:0.057 GP:0.002 RMSEAVG:0.068 NUM:0.068 SynTraiAuc:0.924 RFAcc:0.803                  
[ 8000/12892] LG:-0.297 LD:-0.001 D:0.012 GP:0.001 RMSEAVG:0.068 NUM:0.068 SynTraiAuc:0.924 RFAcc:0.803                 
[ 8500/12892] LG:-0.231 LD:0.030 D:0.041 GP:0.001 RMSEAVG:0.068 NUM:0.068 SynTraiAuc:0.924 RFAcc:0.803                  
[ 9000/12892] LG:-0.260 LD:0.004 D:0.024 GP:0.001 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.924 RFAcc:0.804                  
[ 9500/12892] LG:-0.201 LD:0.018 D:0.032 GP:0.001 RMSEAVG:0.056 NUM:0.056 SynTraiAuc:0.924 RFAcc:0.804                  
[10000/12892] LG:-0.196 LD:0.051